# Solving Mean Field Games (Deep Galerkin)
#### Frederik Kelbel, Imperial College London

In [ ]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb, m, p, H, cat
from DGM import DGMPIASolver, DeepPDESolver
from pdes import HBJ, PDE
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG
from FBSDEs import FBSDESolver
from pdes import FBSDE
import os

In [ ]:
def plot_losses(losses, avg_over=10):
    avgs = np.convolve(losses, np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs)), y=avgs, mode='lines', name="Error at x=0.1"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()

## Linear-Quadratic-Control

Applied Hopf-Cole Transform before solving!!!
Networks were not able to solve problem without transform.
Note: Can we apply other transformations to improve performance.
Note: Can we somehow utilize the Fokker-Planck Equation, i.e. sample from an equation that satisfies it??

In [140]:
class LQG(PDE):
    def __init__(self):
        super().__init__()
        self.l = 5.0
        self.sigma = 0.01
        self.gamma = 0.05
        N = 5
        self.var_dim = N+1 # var = (x, t)
        self.sol_dim = 1
            
        self.equation = lambda u, var: div(u, var[-1]) + (1/2)*self.sigma*Δ(u, var[:-1]) - (1/(2*self.gamma))*torch.sum(D(u, var[:-1])**2, dim=-1, keepdims=True)
        self.domain_func = [(lambda var: self.sampling(var[:-1]) + [5*var[-1]], 128)]
        self.boundary_cond = [lambda u, var: u - (self.l*torch.sum((sum(var[:-1])/N-cat(var[:-1]))**2, dim=-1, keepdims=True)/2)] 
        self.boundary_func = [(lambda var: self.sampling(var[:-1]) + [0*var[-1]+5], 128)]
    
    def sampling(self, var):
        means = (torch.rand((var[-1].shape[0], 1))-0.5)*2
        return [(var[i]-0.5) + means for i in range(len(var))]
    '''
    def sampling(self, var):
        means_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        compactness_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        means = (torch.rand((var[-1].shape[0], self.var_dim-1))-0.5)*2
        compactness = torch.rand((var[-1].shape[0], self.var_dim-1))
        re = [(var[i]-0.5)*compactness.gather(-1, compactness_clusters[:, None, i]) + means.gather(-1, means_clusters[:, None, i]) for i in range(len(var))] 
        return re 
    '''

In [141]:
eq = LQG()
model = MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 1e-3,
    "loss_weights": (1, 4),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "RES",
    "optimiser": "Adam",
    "method": "Galerkin"
}
solver = DeepPDESolver(model, eq)
losses = list(solver.train(400))
plot_losses(losses)

100%|██████████| 400/400 [00:23<00:00, 16.98 it/s]


In [145]:
def plot_game(T):
    n = int(40*T)
    N = eq.var_dim-1
    c_xs = np.zeros((N, n))
    #c_xs[:, 0] = -0.5
    c_xs[:int(2*N/5), 0] = (np.random.randn(int(2*N/5))*0.1 -0.8).clip(-0.7, 0.9)
    c_xs[int(2*N/5):, 0] = (np.random.randn(int(3*N/5))*0.1 +0.5).clip(0.4, 0.6)
    #c_xs[:, 0] = (np.random.randn(N)-0.5).clip(-1, 1)
    uc_xs = np.zeros((N, n))
    uc_xs[:, 0] = c_xs[:, 0]
    dt = 1/n
    ts = [t for t in np.linspace(0, T, n)]
    for i in range(n-1):
        dW = np.sqrt(dt)*np.random.randn(N, 1)
        c = -(1/eq.gamma)*np.expand_dims(solver.D_u(*[c_xs[j, i] for j in range(N)], i*dt), axis=1)
        c_xs[:, None, i+1] = c_xs[:, None, i]  + np.sqrt(eq.sigma)*dW + c*dt
        uc_xs[:, None, i+1] = uc_xs[:, None, i] + np.sqrt(eq.sigma)*dW
        
    return c_xs, uc_xs

In [146]:
c_xs, uc_xs = plot_game(5)

In [147]:
fig = go.Figure(
    data=[go.Scatter(x=c_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
         go.Scatter(x=uc_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)],
    layout=go.Layout(
        xaxis=dict(range=[-1.1, 1.1], autorange=False),
        yaxis=dict(range=[0, 2], autorange=False),
        title="Opinions over time",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 25, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0.1}}])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=c_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
                          go.Scatter(x=uc_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)]) for i in range(c_xs.shape[1])]
)
fig.update_layout(transition = {'duration': 0.1})
fig.show()

## Sznajd Model

We made the substitution of $J = \frac{\phi}{N}$

In [ ]:
class OPINION(PDE):
    def __init__(self):
        super().__init__()
        self.x_d = 0.2
        self.beta = -1
        self.sigma = 0.01
        self.gamma = 0.05
        N = 10
        self.var_dim = N+1 # var = (x, t)
        self.sol_dim = 1       
            
        self.equation = lambda u, var: div(u, var[-1]) + torch.sum((cat(var[:-1]) - self.x_d)**2, dim=-1, keepdims=True)/2 \
                        + torch.sum((self.beta*(1 - cat(var[:-1])**2)
                                     *(torch.mean(cat(var[:-1]), dim=-1, keepdims=True) - cat(var[:-1]))
                                     * D(u, var[:-1])), dim=-1, keepdims=True)\
                        + self.sigma*Δ(u, var[:-1]) \
                        - (1/(2*self.gamma))*torch.sum(D(u, var[:-1])**2, dim=-1, keepdims=True)
        self.domain_func = [(lambda var: self.sampling(var[:-1]) + [5*var[-1]], 128)]
        self.boundary_cond = [lambda u, var: u -torch.sum((cat(var[:-1]) - self.x_d)**2, dim=-1, keepdims=True)/2] 
        self.boundary_func = [(lambda var: self.sampling(var[:-1]) + [0.1*var[-1]+5], 64)]
        
    def sampling(self, var):
        means_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        compactness_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        means = (torch.rand((var[-1].shape[0], self.var_dim-1))-0.5)*2 + self.x_d
        compactness = torch.rand((var[-1].shape[0], self.var_dim-1))
        re = [(var[i]-0.5)*compactness.gather(-1, compactness_clusters[:, None, i]) + means.gather(-1, means_clusters[:, None, i]) for i in range(len(var))] 
        return re

In [ ]:
eq = OPINION()
model = MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 1e-3,
    "loss_weights": (1, 3),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "RESMEAN",
    "optimiser": "Adam",
    "method": "Galerkin"
}
solver = DeepPDESolver(model, eq)
losses = list(solver.train(400))
plot_losses(losses)

In [ ]:
def plot_opinions(T):
    n = int(50*T)
    N = eq.var_dim-1
    def sde_sum(X):
        return eq.beta*(1-X**2)*(sum(X)/N-X)
    c_xs = np.zeros((N, n))
    #c_xs[:, 0] = -0.5
    c_xs[:int(2*N/5), 0] = (np.random.randn(int(2*N/5))*0.2 -0.8).clip(-0.7, 0.9)
    c_xs[int(2*N/5):, 0] = (np.random.randn(int(3*N/5))*0.2 +0.5).clip(0.4, 0.6)
    #c_xs[:, 0] = (np.random.randn(N)-0.5).clip(-1, 1)
    uc_xs = np.zeros((N, n))
    uc_xs[:, 0] = c_xs[:, 0]
    dt = 1/n
    ts = [t for t in np.linspace(0, T, n)]
    for i in range(n-1):
        dW = np.sqrt(dt)*np.random.randn(N, 1)
        c = -(1/(eq.gamma))*np.expand_dims(solver.D_u(*[c_xs[j, i] for j in range(N)], i*dt), axis=1)
        c_xs[:, None, i+1] = c_xs[:, None, i] + sde_sum(c_xs[:, None, i])*dt + np.sqrt(2*eq.sigma)*dW + c*dt
        uc_xs[:, None, i+1] = uc_xs[:, None, i] + sde_sum(uc_xs[:, None, i]) *dt + np.sqrt(2*eq.sigma)*dW
        
    return c_xs, uc_xs

In [ ]:
c_xs, uc_xs = plot_opinions(5)

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=c_xs[:, -1].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)), row=1, col=1)
fig.add_trace(go.Scatter(x=uc_xs[:, -1].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    xaxis_range=[-1.2,1.2],
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [ ]:
fig = go.Figure(
    data=[go.Scatter(x=c_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
         go.Scatter(x=uc_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)],
    layout=go.Layout(
        xaxis=dict(range=[-1.1, 1.1], autorange=False),
        yaxis=dict(range=[0, 2], autorange=False),
        title="Opinions over time",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 25, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0.1}}])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=c_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
                          go.Scatter(x=uc_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)]) for i in range(c_xs.shape[1])]
)
fig.update_layout(transition = {'duration': 0.1})
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-1.1, 1.1, 100)
ys = np.linspace(0, 5, 100)
us_pred = np.array([[solver.u(*([-0.7, -0.7, -0.7, -0.7] + [x for _ in range(eq.var_dim-5)]), y).item() for x in xs] for y in ys])
#us_pred = np.array([[sum([np.abs(x+0.25) for _ in range(10)]).item() for x in xs] for y in ys])
us = us_pred #np.array([[sol(x, t) for x in xs] for t in ts])
x_mesh, y_mesh = np.meshgrid(xs, ys)
fig.add_trace(go.Surface(x=xs, y=ys, z=us, showscale=False), row=1, col=1)
#fig.add_trace(go.Surface(x=ys, y=xs, z=us_pred), row=1, col=2)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="u(x, t)"),
                  scene2 = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="u(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
# one row is printed as x axes
fig.show()

## Hegselmann-Krause

In [ ]:
class BOUNDED_OPINION(PDE):
    def __init__(self):
        super().__init__()
        self.x_d = 0.0
        self.sigma = 0.01
        self.gamma = 0.05
        self.kappa = 0.2
        N = 10
        self.ones = torch.ones(10)
        self.var_dim = N+1 # var = (x, t)
        self.sol_dim = 1    
            
        self.equation = lambda u, var: div(u, var[-1]) + torch.sum((cat(var[:-1]) - self.x_d)**2, dim=-1, keepdims=True)/2 \
                        + torch.sum((1/N)*self.P(cat(var[:-1]))* D(u, var[:-1]), dim=-1, keepdims=True)\
                        + self.sigma*Δ(u, var[:-1]) \
                        - (1/(2*self.gamma))*torch.sum(D(u, var[:-1])**2, dim=-1, keepdims=True)
        self.domain_func = [(lambda var: self.sampling(var[:-1]) + [5*var[-1]], 128)]
        self.boundary_cond = [lambda u, var: u -torch.sum((cat(var[:-1]) - self.x_d)**2, dim=-1, keepdims=True)/2] 
        self.boundary_func = [(lambda var: self.sampling(var[:-1]) + [0.1*var[-1]+5], 64)]
        
    def P(self, X):
        matrix = torch.einsum("bi, j -> bij", X, self.ones) - torch.einsum("bi, j -> bji", X, self.ones)
        return torch.sum((matrix <= self.kappa)*matrix, dim=-1)
        

    def sampling(self, var):
        means_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        compactness_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        means = (torch.rand((var[-1].shape[0], self.var_dim-1))-0.5)*2 + self.x_d
        compactness = torch.rand((var[-1].shape[0], self.var_dim-1))
        return [(var[i]-0.5)*compactness.gather(-1, compactness_clusters[:, None, i]) + means.gather(-1, means_clusters[:, None, i]) for i in range(len(var))] 

In [ ]:
eq = BOUNDED_OPINION()
model = MODEL_CONFIG = {
    "hidden_dim": 64,
    "learning_rate": 1e-3,
    "loss_weights": (1, 2),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "RESMEAN",
    "optimiser": "Adam",
    "method": "Galerkin"
}
solver = DeepPDESolver(model, eq)
losses = list(solver.train(800))
plot_losses(losses)

In [ ]:
def plot_bounded_opinions(T):
    n = int(50*T)
    N = eq.var_dim-1
    def sde_sum(X):
        return (1/N)*eq.P(torch.from_numpy(X).view(1, -1)).numpy().reshape(-1, 1)
    c_xs = np.zeros((N, n))
    #c_xs[:, 0] = -0.5
    c_xs[:int(2*N/5), 0] = (np.random.randn(int(2*N/5))*0.2 -0.8).clip(-0.7, 0.9)
    c_xs[int(2*N/5):, 0] = (np.random.randn(int(3*N/5))*0.2 +0.5).clip(0.4, 0.6)
    #c_xs[:, 0] = (np.random.randn(N)-0.5).clip(-1, 1)
    uc_xs = np.zeros((N, n))
    uc_xs[:, 0] = c_xs[:, 0]
    dt = 1/n
    ts = [t for t in np.linspace(0, T, n)]
    for i in range(n-1):
        dW = np.sqrt(dt)*np.random.randn(N, 1)
        c = -(1/(eq.gamma))*np.expand_dims(solver.D_u(*[c_xs[j, i] for j in range(N)], i*dt), axis=1)
        c_xs[:, None, i+1] = c_xs[:, None, i] + sde_sum(c_xs[:, None, i])*dt + np.sqrt(2*eq.sigma)*dW + c*dt
        uc_xs[:, None, i+1] = uc_xs[:, None, i] + sde_sum(uc_xs[:, None, i]) *dt + np.sqrt(2*eq.sigma)*dW
        
    return c_xs, uc_xs

In [ ]:
c_xs, uc_xs = plot_bounded_opinions(5)

In [ ]:
fig = go.Figure(
    data=[go.Scatter(x=c_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
         go.Scatter(x=uc_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)],
    layout=go.Layout(
        xaxis=dict(range=[-1.6, 1.6], autorange=False),
        yaxis=dict(range=[0, 2], autorange=False),
        title="Opinions over time",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 25, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0.1}}])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=c_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
                          go.Scatter(x=uc_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)]) for i in range(c_xs.shape[1])]
)
fig.update_layout(transition = {'duration': 0.1})
fig.show()

# -----

In [ ]:
class HOPF_OPINION(PDE):
    def __init__(self):
        super().__init__()
        self.x_d = 0.0
        self.beta = -1
        self.sigma = 0.01
        self.gamma = 0.05
        N = 5
        self.var_dim = N+1 # var = (x, t)
        self.sol_dim = 1       
            
        self.equation = lambda u, var: div(u, var[-1]) - torch.sum((cat(var[:-1]) - self.x_d)**2, dim=-1, keepdims=True)/(4*self.gamma*self.sigma) \
                        + torch.sum((self.beta*(1 - cat(var[:-1])**2)
                                     *(torch.mean(cat(var[:-1]), dim=-1, keepdims=True) - cat(var[:-1]))
                                     * D(u, var[:-1])), dim=-1, keepdims=True)\
                        + self.sigma*Δ(u, var[:-1]) + self.sigma*torch.sum(D(u, var[:-1])**2, dim=-1, keepdims=True)
        self.domain_func = [(lambda var: self.sampling(var[:-1]) + [5*var[-1]], 128)]
        self.boundary_cond = [lambda u, var: u] 
        self.boundary_func = [(lambda var: self.sampling(var[:-1]) + [0*var[-1]+5], 64)]
    
    def sampling(self, var):
        means = (torch.rand((var[-1].shape[0], 1))-0.5)*2
        return [(var[i]-0.5) + means for i in range(len(var))]  
    
    '''def sampling(self, var):
        means_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        compactness_clusters = torch.randint(0, self.var_dim-1, size=(var[-1].shape[0], self.var_dim-1))
        means = (torch.rand((var[-1].shape[0], self.var_dim-1))-0.5)*2 + self.x_d
        compactness = torch.rand((var[-1].shape[0], self.var_dim-1))
        re = [(var[i]-0.5)*compactness.gather(-1, compactness_clusters[:, None, i]) + means.gather(-1, means_clusters[:, None, i]) for i in range(len(var))] 
        return re'''

In [ ]:
eq = HOPF_OPINION()
model = MODEL_CONFIG = {
    "hidden_dim": 32,
    "learning_rate": 1e-3,
    "loss_weights": (1, 1),
    "sampling_method": "uniform",
    "lr_decay": 0.98,
    "network_type": "RESMEAN",
    "optimiser": "Adam",
    "method": "Galerkin"
}
solver = DeepPDESolver(model, eq)
losses = list(solver.train(300))
plot_losses(losses)

In [ ]:
def plot_opinions(T):
    n = int(50*T)
    N = eq.var_dim-1
    def sde_sum(X):
        return eq.beta*(1-X**2)*(sum(X)/N-X)
    c_xs = np.zeros((N, n))
    #c_xs[:, 0] = -0.5
    c_xs[:int(2*N/5), 0] = (np.random.randn(int(2*N/5))*0.2 -0.8).clip(-0.7, 0.9)
    c_xs[int(2*N/5):, 0] = (np.random.randn(int(3*N/5))*0.2 +0.5).clip(0.4, 0.6)
    #c_xs[:, 0] = (np.random.randn(N)-0.5).clip(-1, 1)
    uc_xs = np.zeros((N, n))
    uc_xs[:, 0] = c_xs[:, 0]
    dt = 1/n
    ts = [t for t in np.linspace(0, T, n)]
    for i in range(n-1):
        dW = np.sqrt(dt)*np.random.randn(N, 1)
        c = 2*eq.sigma*np.expand_dims(solver.D_u(*[c_xs[j, i] for j in range(N)], i*dt), axis=1)
        c_xs[:, None, i+1] = c_xs[:, None, i] + sde_sum(c_xs[:, None, i])*dt + np.sqrt(2*eq.sigma)*dW + c*dt
        uc_xs[:, None, i+1] = uc_xs[:, None, i] + sde_sum(uc_xs[:, None, i]) *dt + np.sqrt(2*eq.sigma)*dW
        
    return c_xs, uc_xs

In [ ]:
c_xs, uc_xs = plot_opinions(5)

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=c_xs[:, -1].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)), row=1, col=1)
fig.add_trace(go.Scatter(x=uc_xs[:, -1].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    xaxis_range=[-1.2,1.2],
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [ ]:
fig = go.Figure(
    data=[go.Scatter(x=c_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
         go.Scatter(x=uc_xs[:, 0].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)],
    layout=go.Layout(
        xaxis=dict(range=[-1.1, 1.1], autorange=False),
        yaxis=dict(range=[0, 2], autorange=False),
        title="Opinions over time",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 25, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0.1}}])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=c_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Controlled", marker=dict(color='SkyBlue', size=15)),
                          go.Scatter(x=uc_xs[:, i].flatten(), y=np.ones(eq.var_dim-1),
                         mode='markers',
                         name="Uncontrolled", opacity=0.8)]) for i in range(c_xs.shape[1])]
)
fig.update_layout(transition = {'duration': 0.1})
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-1.1, 1.1, 100)
ys = np.linspace(0, 5, 100)
us_pred = np.array([[solver.u(*([-0.7, -0.7, -0.7, -0.7] + [x for _ in range(eq.var_dim-5)]), y).item() for x in xs] for y in ys])
#us_pred = np.array([[sum([np.abs(x+0.25) for _ in range(10)]).item() for x in xs] for y in ys])
us = us_pred #np.array([[sol(x, t) for x in xs] for t in ts])
x_mesh, y_mesh = np.meshgrid(xs, ys)
fig.add_trace(go.Surface(x=xs, y=ys, z=us, showscale=False), row=1, col=1)
#fig.add_trace(go.Surface(x=ys, y=xs, z=us_pred), row=1, col=2)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="u(x, t)"),
                  scene2 = dict(
                    xaxis_title="x",
                    yaxis_title="t",
                    zaxis_title="u(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
# one row is printed as x axes
fig.show()